#### Monday, December 18, 2023

Trying this in the conda container hfpt_Dec14 ... I'm going to skip all the code that uses the HuggingFaceHub and so straight to using the local models.

Ok. The local code still works. Nice.

#### Tuesday, December 5, 2023

[LangChain - Using Hugging Face Models locally (code walkthrough)](https://www.youtube.com/watch?v=Kn7SX2Mx_Jk)

https://colab.research.google.com/drive/1h2505J5H4Y9vngzPD08ppf1ga8sWxLvZ?usp=sharing#scrollTo=VkVTT54xNq8T

I was able to step through most of this notebook.


In [ ]:
# !pip -q install langchain huggingface_hub transformers sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.2/358.2 KB 17.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 63.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 27.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 5.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.3 MB/s eta 0:00:00


## HuggingFace

There are two Hugging Face LLM wrappers, one for a local pipeline and one for a model hosted on Hugging Face Hub. Note that these wrappers only work for models that support the following tasks: text2text-generation, text-generation


In [1]:
from getpass import getpass

# enter your api key
HUGGINGFACEHUB_API_TOKEN = getpass("Enter your HuggingFace API Token")

In [2]:
import os

os.environ['HUGGINGFACEHUB_API_TOKEN'] = HUGGINGFACEHUB_API_TOKEN

## Use the HuggingFaceHub

In [3]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [4]:
llm_chain = LLMChain(prompt=prompt,
                     llm=HuggingFaceHub(repo_id="google/flan-t5-xl",
                                        model_kwargs={"temperature":0,
                                                      "max_length":64}))

/usr/local/lib/python3.8/dist-packages/huggingface_hub/utils/_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '0.19.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


In [5]:
# This ALWAYS times out! ...
question = "What is the capital of France?"

print(llm_chain.run(question))

# 5m 34.3s

ValueError: Error raised by inference API: Model google/flan-t5-xl time out

In [ ]:
# I am not going to run this because I bet it too will time out ...
question = "What area is best for growing wine in France?"

print(llm_chain.run(question))

The best area for growing wine in France is the Loire Valley. The Loire Valley is located in the south of France. The area of France that is best for growing wine is the Loire Valley. The final answer: Loire Valley.


## BlenderBot

Doesn't work on the Hub

In [ ]:
# I am not going to run this cell ... 
blenderbot_chain = LLMChain(prompt=prompt,
                     llm=HuggingFaceHub(repo_id="facebook/blenderbot-1B-distill",
                                        model_kwargs={"temperature":0,
                                                      "max_length":64}))

ValidationError: ignored

In [ ]:
# question = "What is the capital of France?"
# question = "What area is best for growing wine in France?"

# print(blenderbot_chain = LLMChain(prompt=prompt,
# .run(question))

## With Local model from HF

### Why would you want to use local mode?

- fine-tuned models
- GPU hosted etc
- some models only work locally

In [1]:
# only target the 4090 ...
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [2]:
# !ls /home/rob/Data2/huggingface/transformers
!ls /root/.cache/huggingface/hub

models--HuggingFaceH4--zephyr-7b-beta
models--bert-base-uncased
models--cross-encoder--ms-marco-MiniLM-L-6-v2
models--deepset--roberta-base-squad2
models--distilbert-base-uncased-distilled-squad
models--facebook--blenderbot-1B-distill
models--google--flan-t5-large
models--gpt2-medium
models--meta-llama--Llama-2-13b-hf
models--my_model--language_model.bin
models--sentence-transformers--all-MiniLM-L6-v2
models--sentence-transformers--clip-ViT-B-32
models--sentence-transformers--multi-qa-mpnet-base-dot-v1
version.txt


In [7]:
# load in the target model to this container ...

## T5-Flan - Encoder-Decoder

In [2]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM


In [3]:
model_id = 'google/flan-t5-large'# go for a smaller model if you dont have the VRAM
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [4]:
%%time
model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True)

# 4.8s 1710 MiB VRAM

# download time ...
# CPU times: user 19.2 s, sys: 17.2 s, total: 36.4 s
# Wall time: 43min 38s

CPU times: user 7.12 s, sys: 1.06 s, total: 8.18 s
Wall time: 2.77 s


In [6]:
# Backup models--google--flan-t5-large
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--google--flan-t5-large /home/rob/Data3/huggingface/transformers
# Successfully copied 3.14GB to /home/rob/Data3/huggingface/transformers

In [3]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [6]:
# from langchain.llms import HuggingFacePipeline
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

# model_id = 'google/flan-t5-large'# go for a smaller model if you dont have the VRAM
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id, load_in_8bit=True)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)


In [7]:
# This puts a negligible load on the GPU
print(local_llm('What is the capital of France? '))

# 0.7s

paris


In [8]:
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )

question = "What is the capital of England?"

print(llm_chain.run(question))

# 2.1s

The capital of England is London. London is the capital of England. So the answer is London.


## GPT2-medium - Decoder Only Model

microsoft/DialoGPT-large

In [9]:
!ls /root/.cache/huggingface/hub

models--HuggingFaceH4--zephyr-7b-beta
models--bert-base-uncased
models--cross-encoder--ms-marco-MiniLM-L-6-v2
models--deepset--roberta-base-squad2
models--distilbert-base-uncased-distilled-squad
models--google--flan-t5-large
models--meta-llama--Llama-2-13b-hf
models--my_model--language_model.bin
models--sentence-transformers--all-MiniLM-L6-v2
models--sentence-transformers--clip-ViT-B-32
models--sentence-transformers--multi-qa-mpnet-base-dot-v1
version.txt


In [ ]:
# docker cp /home/rob/Data3/huggingface/transformers/models--gpt2-medium  c8324b70601d://root/.cache/huggingface/hub
# Successfully copied 1.52GB to c8324b70601d://root/.cache/huggingface/hub


In [10]:
!ls /root/.cache/huggingface/hub

models--HuggingFaceH4--zephyr-7b-beta
models--bert-base-uncased
models--cross-encoder--ms-marco-MiniLM-L-6-v2
models--deepset--roberta-base-squad2
models--distilbert-base-uncased-distilled-squad
models--google--flan-t5-large
models--gpt2-medium
models--meta-llama--Llama-2-13b-hf
models--my_model--language_model.bin
models--sentence-transformers--all-MiniLM-L6-v2
models--sentence-transformers--clip-ViT-B-32
models--sentence-transformers--multi-qa-mpnet-base-dot-v1
version.txt


In [10]:
# This loads the GPT-2 model to the CPU and uses it to generate text.
# from transformers import pipeline, set_seed
# generator = pipeline('text-generation', model='gpt2-medium')

In [4]:
model_id = "gpt2-medium"
# tokenizer = AutoTokenizer.from_pretrained(model_id)

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [5]:
# This loads the model to the CPU ... 
# model = AutoModelForCausalLM.from_pretrained(model_id)

# This works! VRAM 896 MiB
# model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True)

# This works! VRAM 1780 MiB
# model = AutoModelForCausalLM.from_pretrained(model_id, device_map='cuda')

# This works! VRAM 1780 MiB
# model = AutoModelForCausalLM.from_pretrained(model_id, device_map=device)

# This loads the model to CPU.
# model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16)

# This worked in the 'Intel_neural_chat_7b_v3_1_+_4_8bit_Basics.ipynb' notebook for the ''Intel/neural-chat-7b-v3-1' model ...
# OK! This DOES work! ... 896 MiB VRAM
model = AutoModelForCausalLM.from_pretrained(model_id, load_in_8bit=True, torch_dtype=torch.float16)



In [ ]:
# Backup "gpt2-medium"
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--gpt2-medium /home/rob/Data3/huggingface/transformers
# Successfully copied 1.52GB to /home/rob/Data3/huggingface/transformers

In [7]:
# model_id = "gpt2-medium"
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForCausalLM.from_pretrained(model_id)

pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [8]:
# This sucks up %100 of all 8 CPU Cores!!
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )

question = "What is the capital of France?"

print(llm_chain.run(question))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.




Let's think French French

Capital

The capital is made of 1/250 the amount of French. (1/3 of the total population of France is French and 1/100 of it is foreigners.) In a certain economic context, the amount of French in France depends solely on the economy. The average of the economic activity of France becomes a capital. But the "money


## BlenderBot - Encoder-Decoder

In [12]:
from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

In [ ]:
# docker cp /home/rob/Data3/huggingface/transformers/models--facebook--blenderbot-1B-distill  c8324b70601d://root/.cache/huggingface/hub
# Successfully copied 2.88GB to c8324b70601d://root/.cache/huggingface/hub

In [13]:
model_id = 'facebook/blenderbot-1B-distill'
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [14]:
# This loads the model to the CPU ... not the GPU ... we want the GPU
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [ ]:
# Backup 'facebook/blenderbot-1B-distill'
# docker cp c9b676310ea0://home/rob/Data2/huggingface/transformers/models--facebook--blenderbot-1B-distill /home/rob/Data3/huggingface/transformers
# Successfully copied 2.88GB to /home/rob/Data3/huggingface/transformers

In [15]:
# from langchain.llms import HuggingFacePipeline
# import torch
# from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM

# model_id = 'facebook/blenderbot-1B-distill'
# tokenizer = AutoTokenizer.from_pretrained(model_id)
# model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

pipe = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)

In [16]:
# This too sucks up %100 of 8 CPU Cores!!
llm_chain = LLMChain(prompt=prompt,
                     llm=local_llm
                     )

question = "What area is best for growing wine in France?"

print(llm_chain.run(question))

 I'm not sure, but I do know that France is one of the largest producers of wine in the world.


## SentenceTransformers

In [ ]:
# docker cp /home/rob/Data3/huggingface/transformers/models--facebook--blenderbot-1B-distill  c8324b70601d://root/.cache/huggingface/hub
# Successfully copied 2.88GB to c8324b70601d://root/.cache/huggingface/hub

In [23]:
!ls /root/.cache/huggingface/hub

models--HuggingFaceH4--zephyr-7b-beta
models--bert-base-uncased
models--cross-encoder--ms-marco-MiniLM-L-6-v2
models--deepset--roberta-base-squad2
models--distilbert-base-uncased-distilled-squad
models--facebook--blenderbot-1B-distill
models--google--flan-t5-large
models--gpt2-medium
models--meta-llama--Llama-2-13b-hf
models--my_model--language_model.bin
models--sentence-transformers--all-MiniLM-L6-v2
models--sentence-transformers--clip-ViT-B-32
models--sentence-transformers--multi-qa-mpnet-base-dot-v1
version.txt


In [21]:
# To where is this thing loaded??
from langchain.embeddings import HuggingFaceEmbeddings

model_name = "sentence-transformers/all-mpnet-base-v2"

hf = HuggingFaceEmbeddings(model_name=model_name)

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

train_script.py:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

To where is the above "sentence-transformers/all-mpnet-base-v2" downloaded?

In [26]:
!ls ~/.cache/huggingface/hub

066329cdbf0bb3f3816e6f84e2d4876143fd4e26a11ec3ed7c9837e33d9fe41e.30cb82e2121eb5ce659a35437f3a58bdceac7b5cd7c0aaaa1f10345719259ba9
066329cdbf0bb3f3816e6f84e2d4876143fd4e26a11ec3ed7c9837e33d9fe41e.30cb82e2121eb5ce659a35437f3a58bdceac7b5cd7c0aaaa1f10345719259ba9.json
066329cdbf0bb3f3816e6f84e2d4876143fd4e26a11ec3ed7c9837e33d9fe41e.30cb82e2121eb5ce659a35437f3a58bdceac7b5cd7c0aaaa1f10345719259ba9.lock
models--llama2-13b-journal-finetune--checkpoint-500
models--robkayinto--bloomz-560m_PROMPT_TUNING_CAUSAL_LM
models--robkayinto--roberta-large-lora-token-classification
models--robkayinto--t5-large_PREFIX_TUNING_SEQ2SEQ
models--robkayinto--vit-base-patch16-224-in21k-finetuned-lora-food101
models--stevhliu--roberta-large-lora-token-classification


In [57]:
# !find ~/.cache/huggingface '*net*'

In [50]:
!ls ~/.cache/huggingface/token

/root/.cache/huggingface/token


In [24]:
hf.embed_query('this is an embedding')

[0.010657303035259247,
 -0.09967268258333206,
 -0.02696710266172886,
 0.0653177872300148,
 0.02100498043000698,
 0.04262344911694527,
 0.011534187011420727,
 -0.00622932193800807,
 0.05175819620490074,
 0.0073067620396614075,
 0.02135349251329899,
 0.04269150644540787,
 0.02314387448132038,
 0.009952746331691742,
 0.056463126093149185,
 -0.0613798052072525,
 0.0527438186109066,
 0.024683993309736252,
 -0.013267739675939083,
 -0.007051229476928711,
 0.02665632963180542,
 -0.005913537926971912,
 0.0040975091978907585,
 0.03841238096356392,
 -0.014230655506253242,
 0.023023536428809166,
 -0.007326608989387751,
 -0.03562534227967262,
 -0.017934108152985573,
 -0.013930202461779118,
 0.011977520771324635,
 -0.007365986239165068,
 0.02445153333246708,
 -0.06637249886989594,
 1.567764797982818e-06,
 0.018217220902442932,
 0.0019748881459236145,
 -0.018329335376620293,
 -0.014930757693946362,
 -0.00539341289550066,
 -0.011222339235246181,
 0.01579292304813862,
 -0.027141854166984558,
 -0.015628

In [25]:
hf.embed_documents(['this is an embedding','this another embedding'])

[[0.010657299309968948,
  -0.09967265278100967,
  -0.02696710079908371,
  0.0653177797794342,
  0.021004967391490936,
  0.04262346029281616,
  0.01153416745364666,
  -0.00622935127466917,
  0.051758211106061935,
  0.007306776475161314,
  0.021353481337428093,
  0.04269150644540787,
  0.023143889382481575,
  0.009952714666724205,
  0.05646311119198799,
  -0.06137977913022041,
  0.0527438148856163,
  0.024684008210897446,
  -0.013267725706100464,
  -0.00705121923238039,
  0.02665632776916027,
  -0.005913522094488144,
  0.004097490105777979,
  0.038412392139434814,
  -0.014230655506253242,
  0.023023530840873718,
  -0.007326585240662098,
  -0.035625357180833817,
  -0.017934130504727364,
  -0.013930218294262886,
  0.011977512389421463,
  -0.007365989498794079,
  0.024451525881886482,
  -0.06637248396873474,
  1.5677645706091425e-06,
  0.01821722276508808,
  0.001974881859496236,
  -0.018329354003071785,
  -0.014930780045688152,
  -0.005393416155129671,
  -0.011222345754504204,
  0.01579293

In [ ]:


hf = HuggingFaceHubEmbeddings(
    repo_id=model_name,
    task="feature-extraction",
    # huggingfacehub_api_token="my-api-key",
)